### Earthquake Predictor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("Earthquake.csv")
data.columns



Index(['Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status'],
      dtype='object')

In [3]:
data = data[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]
data.head()

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,02-01-1965,13.44.18,19.246,145.616,131.6,6.0
1,04-01-1965,11.29.49,1.863,127.352,80.0,5.8
2,05-01-1965,18.05.58,-20.579,-173.972,20.0,6.2
3,08-01-1965,18.49.43,-59.076,-23.557,15.0,5.8
4,09-01-1965,13.32.50,11.938,126.427,15.0,5.8


In [4]:
import datetime
import time

timestamp = []
for d, t in zip(data['Date'], data['Time']):
    try:
        ts = datetime.datetime.strptime(d+' '+t, '%m-%d-%Y %H.%M.%S')
        timestamp.append(time.mktime(ts.timetuple()))
    except:
        # print('ValueError')
        timestamp.append('ValueError')
timeStamp = pd.Series(timestamp)
data['Timestamp'] = timeStamp.values
final_data = data.drop(['Date', 'Time'], axis=1)
final_data = final_data[final_data.Timestamp != 'ValueError']
final_data.head()

,Latitude,Longitude,Depth,Magnitude,Timestamp
1456,-29.400,-177.169,35.0,5.6,42060.0
1457,24.185,102.543,11.3,7.1,7817441.0
1458,23.984,102.732,15.0,5.9,10390750.0
1459,-9.583,151.493,15.0,6.3,13046754.0
1460,15.785,-59.808,36.7,6.0,15647174.0


In [5]:
# from mpl_toolkits.basemap import Basemap

# m = Basemap(projection='mill',llcrnrlat=-80,urcrnrlat=80, llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')

# longitudes = data["Longitude"].tolist()
# latitudes = data["Latitude"].tolist()
# #m = Basemap(width=12000000,height=9000000,projection='lcc',
#             #resolution=None,lat_1=80.,lat_2=55,lat_0=80,lon_0=-107.)
# x,y = m(longitudes,latitudes)

# fig = plt.figure(figsize=(12,10))
# plt.title("All affected areas")
# m.plot(x, y, "o", markersize = 2, color = 'blue')
# m.drawcoastlines()
# m.fillcontinents(color='coral',lake_color='aqua')
# m.drawmapboundary()
# m.drawcountries()
# plt.show()

In [6]:
X = final_data[['Timestamp', 'Latitude', 'Longitude']]
y = final_data[['Magnitude', 'Depth']]


In [7]:
X.head(1)

,Timestamp,Latitude,Longitude
1456,42060.0,-29.4,-177.169


In [8]:
X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32')



In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

(6899, 3) (1725, 3) (6899, 2) (1725, 3)


In [10]:
X_train

array([[ 9.4428070e+08, -2.0035999e+01,  1.6857500e+02],
       [ 1.2443666e+09,  5.0435001e+01,  1.7699200e+02],
       [ 3.0807462e+08, -5.4326000e+01, -1.3282100e+02],
       ...,
       [ 9.7881216e+08, -4.3119998e+00,  1.5295599e+02],
       [ 1.8478259e+08,  6.9889998e+00, -1.0427500e+02],
       [ 1.2864973e+09,  3.9405998e+01,  1.4314799e+02]], dtype=float32)

In [11]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

# neurons = [16, 64, 128, 256]
neurons = [16]
# batch_size = [10, 20, 50, 100]
batch_size = [10]
epochs = [10]
# activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'exponential']
activation = ['sigmoid', 'relu']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizer = ['SGD', 'Adadelta']
loss = ['squared_hinge']

param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs, activation=activation, optimizer=optimizer, loss=loss)


In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.800000 using {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.600000 (0.489898) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.400000 (0.489898) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}
0.800000 (0.400000) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.000000 (0.000000) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}


In [15]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(3,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=10, epochs=10, verbose=1, validation_data=(X_test, y_test))

[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Epoch 1/10
690/690 [==============================] - 2s 3ms/step - loss: 0.5026 - accuracy: 0.9841 - val_loss: 0.5049 - val_accuracy: 0.9797
Epoch 2/10
690/690 [==============================] - 2s 2ms/step - loss: 0.5026 - accuracy: 0.9841 - val_loss: 0.5049 - val_accuracy: 0.9797
Epoch 3/10
690/690 [==============================] - 2s 2ms/step - loss: 0.5026 - accuracy: 0.9841 - val_loss: 0.5049 - val_accuracy: 0.9797
Epoch 4/10
690/690 [==============================] - 2s 2ms/step - loss: 0.5026 - accuracy: 0.9841 - val_loss: 0.5049 - val_accuracy: 0.9797
Epoch 5/10
690/690 [==============================] - 2s 2ms/step - loss: 0.5026 - accuracy: 0.9841 - val_loss: 0.5049 - val_accuracy: 0.9797
Epoch 6/10
690/690 [==============================] - 2s 2ms/step - loss: 0.5026 - accuracy: 0.9841 - val_loss: 0.5049 - val_accuracy: 0.9797
Epoch 7/10
690/690 [==============================] - 2s 2ms/step - loss: 0.5026 - accuracy: 0.9841 - val_loss: 0.5049 - val_accuracy: 0.9797
Epoch 

In [16]:
model.save("Earthquake")
model.save_weights("weights.h5")



INFO:tensorflow:Assets written to: Earthquake\assets
